# **Mount Google Drive & install libraries**

In [18]:
# Mount Google Drive

from google.colab import drive
drive.mount("/content/drive")

%cd /content/drive/My Drive/Coding Projects

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Coding Projects


In [19]:
# Import needed libraries
!pip install python-docx
print("Installed python-docx successfully!")
print("")


import pandas as pd
import uuid
print("Imported libraries successfully!")

Installed python-docx successfully!

Imported libraries successfully!


# **Preparing the data**

## **Cleaning Associates data**

In [20]:
# Cleaning data for Associates

df_assocs_raw = pd.read_excel("sample_dataset.xlsx", sheet_name="Associates") # Read the spreadsheet
df_assocs_raw = df_assocs_raw[df_assocs_raw["employee_id"].notnull()] # Drop null rows

# Fix the datatypes
df_assocs_raw["report_year"] = df_assocs_raw["report_year"].astype(int)
df_assocs_raw["report_month"] = df_assocs_raw["report_month"].astype(int)
# df_assocs_raw["report_date"] = df_assocs_raw["report_date"].astype(int)
df_assocs_raw["employee_id"] = df_assocs_raw["employee_id"].astype(str).str.replace('.0','')
df_assocs_raw["agent_name"] = df_assocs_raw["agent_name"].astype(str)
df_assocs_raw["account"] = df_assocs_raw["account"].astype(str)
df_assocs_raw["team_lead"] = df_assocs_raw["team_lead"].astype(str)
df_assocs_raw["metric_1_score"] = pd.to_numeric(df_assocs_raw["metric_1_score"])
df_assocs_raw["metric_2_score"] = pd.to_numeric(df_assocs_raw["metric_2_score"]) # No need cause its already numeric
df_assocs_raw["metric_3_score"] = pd.to_numeric(df_assocs_raw["metric_3_score"])
df_assocs_raw["metric_1_incentive_amt"] = pd.to_numeric(df_assocs_raw["metric_1_incentive_amt"])
df_assocs_raw["metric_2_incentive_amt"] = pd.to_numeric(df_assocs_raw["metric_2_incentive_amt"])
df_assocs_raw["metric_3_incentive_amt"] = pd.to_numeric(df_assocs_raw["metric_3_incentive_amt"])
df_assocs_raw["total_incentive_amt"] = pd.to_numeric(df_assocs_raw["total_incentive_amt"])

# Check if the datatypes are correct
df_assocs_raw.dtypes

,0
report_year,int64
report_month,int64
report_date,datetime64[ns]
employee_id,object
agent_name,object
account,object
team_lead,object
metric_1_score,float64
metric_2_score,float64
metric_3_score,float64


In [21]:
df_assocs = df_assocs_raw[["report_date",
                           "employee_id",
                           "agent_name",
                           "account",
                           "team_lead",
                           "metric_1_score",
                           "metric_2_score",
                           "metric_3_score",
                           "metric_1_incentive_amt",
                           "metric_2_incentive_amt",
                           "metric_3_incentive_amt",
                           "total_incentive_amt"
                          ]]

account_order = ["Account 1", # List the accounts in specific order; this is required so you can determine ranking of agent per account; to optimize, explore window ranking
                 "Account 2",
                 "Account 3",
                 "Account 4",
                 "Account 5",
                 "Account 6",
                 "Account 7",
                 "Account 8",
                 "Account 9",
                 "Account 10",
                 "Account 11",
                 "Account 12",
                 "Account 13",
                 "Account 14",
                 "Account 15",
                 "Account 16",
                 "Account 17"]
df_assocs["account"] = pd.Categorical(df_assocs["account"], categories=account_order, ordered=True) # Ssetting "Account" as categorical preserves its order in the list

df_assocs_summary_table = df_assocs.groupby("account").agg({"employee_id": pd.Series.nunique, "total_incentive_amt": "sum"}).reset_index()
df_assocs_summary_table

<ipython-input-21-efe228d722ba>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assocs["account"] = pd.Categorical(df_assocs["account"], categories=account_order, ordered=True) # Ssetting "Account" as categorical preserves its order in the list
<ipython-input-21-efe228d722ba>:34: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_assocs_summary_table = df_assocs.groupby("account").agg({"employee_id": pd.Series.nunique, "total_incentive_amt": "sum"}).reset_index()


,account,employee_id,total_incentive_amt
0,Account 1,40,130416
1,Account 2,27,84846
2,Account 3,26,82764
3,Account 4,68,191623
4,Account 5,25,68071
5,Account 6,7,20956
6,Account 7,13,41389
7,Account 8,31,88119
8,Account 9,42,125675
9,Account 10,6,20221


In [22]:
# Total incentive amount to be paid out to Associates
assocs_total_incentive_amt = df_assocs["total_incentive_amt"].sum()
assocs_total_incentive_amt

1531429

In [23]:
# Sort the columns to prepare for ranking
df_assocs_sorted = df_assocs.sort_values(by=["account", "team_lead", "total_incentive_amt", "metric_1_score", "metric_2_score", "metric_3_score"], ascending=[True, True, False, False, False, True])

# Rank the associates
df_assocs_sorted["rank"] = df_assocs_sorted.groupby(df_assocs.columns[[3, 4]].tolist()).cumcount() + 1
df_assocs_sorted["rank"] = df_assocs_sorted["rank"].astype(int)

df_assocs_sorted["metric_1_score"] = df_assocs_sorted["metric_1_score"].fillna(-1).apply(lambda x: f"{(x * 100):.2f}%") # Formats the attendance column to "XX.XX%" and fills nulls with -1
df_assocs_sorted.loc[df_assocs_sorted["metric_1_score"] == "-100.00%", "metric_1_score"] = "N/A" # Changes -1 columns to N/A; https://stackoverflow.com/questions/31511997/pandas-dataframe-replace-all-values-in-a-column-based-on-condition#:~:text=In%20%5B41%5D%3A-,df.loc%5Bdf%5B%27First%20Season%27%5D%20%3E%201990%2C%20%27First%20Season%27%5D%20%3D%201,-df%0A%0AOut%5B

df_assocs_sorted["metric_2_score"] = df_assocs_sorted["metric_2_score"].fillna(-1).apply(lambda x: f"{(x * 100):.2f}%")
df_assocs_sorted.loc[df_assocs_sorted["metric_2_score"] == "-100.00%", "metric_2_score"] = "N/A"

df_assocs_sorted["metric_3_score"] = df_assocs_sorted["metric_3_score"].fillna(0).astype(int)
df_assocs_sorted.loc[df_assocs["metric_3_score"] == 0, "metric_3_score"] = "N/A"

df_assocs_sorted["total_incentive_amt"] = df_assocs_sorted["total_incentive_amt"].apply(lambda x: f"₱ {x:,.0f}")

df_assocs_sorted

<ipython-input-23-301e93615623>:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_assocs_sorted["rank"] = df_assocs_sorted.groupby(df_assocs.columns[[3, 4]].tolist()).cumcount() + 1
<ipython-input-23-301e93615623>:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_assocs_sorted.loc[df_assocs["metric_3_score"] == 0, "metric_3_score"] = "N/A"


,report_date,employee_id,agent_name,account,team_lead,metric_1_score,metric_2_score,metric_3_score,metric_1_incentive_amt,metric_2_incentive_amt,metric_3_incentive_amt,total_incentive_amt,rank
275,2024-08-31,E276,Employee 276,Account 1,TEAM LEAD NAME 25,100.00%,87.07%,511,1279,1169,1679,"₱ 4,127",1
286,2024-08-31,E287,Employee 287,Account 1,TEAM LEAD NAME 25,92.84%,93.80%,536,1908,634,1565,"₱ 4,107",2
254,2024-08-31,E255,Employee 255,Account 1,TEAM LEAD NAME 25,100.00%,99.42%,442,1205,1726,1061,"₱ 3,992",3
293,2024-08-31,E294,Employee 294,Account 1,TEAM LEAD NAME 25,100.00%,97.67%,517,1674,539,1740,"₱ 3,953",4
282,2024-08-31,E283,Employee 283,Account 1,TEAM LEAD NAME 25,92.04%,95.16%,521,1891,505,889,"₱ 3,285",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,2024-08-31,E311,Employee 311,Account 16,TEAM LEAD NAME 32,100.00%,96.00%,496,1018,40,96,"₱ 1,154",14
298,2024-08-31,E299,Employee 299,Account 17,TEAM LEAD NAME 33,100.00%,100.00%,440,1311,1380,1721,"₱ 4,412",1
364,2024-08-31,E365,Employee 365,Account 17,TEAM LEAD NAME 33,81.82%,100.00%,489,1288,1188,1178,"₱ 3,654",2
297,2024-08-31,E298,Employee 298,Account 17,TEAM LEAD NAME 33,100.00%,100.00%,487,1193,1161,768,"₱ 3,122",3


In [24]:
# Double checking the datatypes
df_assocs_sorted.dtypes

,0
report_date,datetime64[ns]
employee_id,object
agent_name,object
account,category
team_lead,object
metric_1_score,object
metric_2_score,object
metric_3_score,object
metric_1_incentive_amt,int64
metric_2_incentive_amt,int64


## **Cleaning Team Leads data**

In [25]:
# Cleaning data for Team Leads

df_leads_raw = pd.read_excel("sample_dataset.xlsx", sheet_name="Team Leads") # Read the spreadsheet
df_leads_raw = df_leads_raw[df_leads_raw["employee_id"].notnull()] # Drop null rows

# Fix the datatypes
df_leads_raw["report_year"] = df_leads_raw["report_year"].astype(int)
df_leads_raw["report_month"] = df_leads_raw["report_month"].astype(int)
# df_leads_raw["report_date"] = df_leads_raw["report_date"]
df_leads_raw["employee_id"] = df_leads_raw["employee_id"].astype(str).str.replace('.0','')
df_leads_raw["team_lead_name"] = df_leads_raw["team_lead_name"].astype(str)
df_leads_raw["account"] = df_leads_raw["account"].astype(str)
df_leads_raw["metric_1_score"] = df_leads_raw["metric_1_score"]
df_leads_raw["metric_2_score"] = df_leads_raw["metric_2_score"]
df_leads_raw["metric_3_score"] = df_leads_raw["metric_3_score"]
df_leads_raw["metric_4_score"] = df_leads_raw["metric_4_score"]
df_leads_raw["metric_5_score"] = df_leads_raw["metric_5_score"]
df_leads_raw["metric_1_incentive_amt"] = df_leads_raw["metric_1_incentive_amt"]
df_leads_raw["metric_2_incentive_amt"] = df_leads_raw["metric_2_incentive_amt"]
df_leads_raw["metric_3_incentive_amt"] = df_leads_raw["metric_3_incentive_amt"]
df_leads_raw["metric_4_incentive_amt"] = df_leads_raw["metric_4_incentive_amt"]
df_leads_raw["metric_5_incentive_amt"] = df_leads_raw["metric_5_incentive_amt"]
df_leads_raw["total_incentive_amt"] = df_leads_raw["total_incentive_amt"]

# df_leads_raw.dtypes
df_leads_raw.head()

,report_year,report_month,report_date,employee_id,team_lead_name,account,metric_1_score,metric_2_score,metric_3_score,metric_4_score,metric_5_score,metric_1_incentive_amt,metric_2_incentive_amt,metric_3_incentive_amt,metric_4_incentive_amt,metric_5_incentive_amt,total_incentive_amt
0,2024,8,2024-08-31,TL1,TEAM LEAD NAME 1,Account 4,1.0000,0.090909,0.818182,0.545455,1.000000,1505,40,1146,244,695,3630
1,2024,8,2024-08-31,TL2,TEAM LEAD NAME 2,Account 4,1.0000,0.000000,0.600000,1.000000,0.700000,1640,441,868,911,1219,5079
2,2024,8,2024-08-31,TL3,TEAM LEAD NAME 3,Account 4,1.0000,0.000000,0.666667,0.500000,0.750000,573,402,266,1971,463,3675
3,2024,8,2024-08-31,TL4,TEAM LEAD NAME 4,Account 4,1.0000,0.000000,0.833333,0.750000,0.750000,454,144,1854,1076,1448,4976
4,2024,8,2024-08-31,TL5,TEAM LEAD NAME 5,Account 4,0.9545,0.000000,0.636364,0.818182,0.727273,882,252,989,1918,1289,5330


In [26]:
df_leads = df_leads_raw[["report_date",
                         "employee_id",
                         "team_lead_name",
                         "account",
                         "metric_1_score",
                         "metric_2_score",
                         "metric_3_score",
                         "metric_4_score",
                         "metric_5_score",
                         "metric_1_incentive_amt",
                         "metric_2_incentive_amt",
                         "metric_3_incentive_amt",
                         "metric_4_incentive_amt",
                         "metric_5_incentive_amt",
                         "total_incentive_amt"
                       ]]

account_order = ["Account 1", # List the accounts in specific order; this is required so you can determine ranking of agent per account; to optimize, explore window ranking
                 "Account 2",
                 "Account 3",
                 "Account 4",
                 "Account 5",
                 "Account 6",
                 "Account 7",
                 "Account 8",
                 "Account 9",
                 "Account 10",
                 "Account 11",
                 "Account 12",
                 "Account 13",
                 "Account 14",
                 "Account 15",
                 "Account 16",
                 "Account 17"]
df_leads["account"] = pd.Categorical(df_leads["account"], categories=account_order, ordered=True) # Setting "Account" as categorical preserves its order in the list

df_leads_summary_table = df_leads.groupby("account").agg({"employee_id": pd.Series.nunique, "total_incentive_amt": "sum"}).reset_index()
df_leads_summary_table

<ipython-input-26-39699e65e140>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_leads["account"] = pd.Categorical(df_leads["account"], categories=account_order, ordered=True) # Setting "Account" as categorical preserves its order in the list
<ipython-input-26-39699e65e140>:37: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_leads_summary_table = df_leads.groupby("account").agg({"employee_id": pd.Series.nunique, "total_incentive_amt": "sum"}).reset_index()


,account,employee_id,total_incentive_amt
0,Account 1,3,12817
1,Account 2,3,17461
2,Account 3,2,10489
3,Account 4,6,27230
4,Account 5,1,7193
5,Account 6,1,5935
6,Account 7,1,3775
7,Account 8,3,13171
8,Account 9,5,22681
9,Account 10,1,4914


In [27]:
tls_total_incentive_amt = df_leads["total_incentive_amt"].sum()
tls_total_incentive_amt

print(tls_total_incentive_amt)
print(assocs_total_incentive_amt + tls_total_incentive_amt) # Total incentives to be paid out Associates + Team Leads

245247
1776676


In [28]:
# Sort the columns to prepare for ranking
df_leads_sorted = df_leads.sort_values(by=["account",
                                           "total_incentive_amt",
                                           "metric_2_score",
                                           "metric_3_score",
                                           "metric_4_score",
                                           "metric_5_score",
                                           "metric_1_score"], ascending=[True, False, True, False, False, False, False])

# Rank the team leads
df_leads_sorted["rank"] = df_leads_sorted.groupby(df_leads.columns[[3]].tolist()).cumcount() + 1
# df_leads_sorted["rank"] = df_leads_sorted["rank"].astype(int)

df_leads_sorted["metric_1_score"] = df_leads_sorted["metric_1_score"].fillna(-1).apply(lambda x: f"{(x * 100):.2f}%") # Formats the column to "XX.XX%" and fills nulls with -1
df_leads_sorted.loc[df_leads_sorted["metric_1_score"] == "-100.00%", "metric_1_score"] = "N/A" # Changes -1 columns to N/A; https://stackoverflow.com/questions/31511997/pandas-dataframe-replace-all-values-in-a-column-based-on-condition#:~:text=In%20%5B41%5D%3A-,df.loc%5Bdf%5B%27First%20Season%27%5D%20%3E%201990%2C%20%27First%20Season%27%5D%20%3D%201,-df%0A%0AOut%5B

df_leads_sorted["metric_2_score"] = df_leads_sorted["metric_2_score"].fillna(-1).apply(lambda x: f"{(x * 100):.2f}%")
df_leads_sorted.loc[df_leads_sorted["metric_2_score"] == "-100.00%", "metric_2_score"] = "N/A"

df_leads_sorted["metric_3_score"] = df_leads_sorted["metric_3_score"].fillna(-1).apply(lambda x: f"{(x * 100):.2f}%")
df_leads_sorted.loc[df_leads_sorted["metric_3_score"] == "-100.00%", "metric_3_score"] = "N/A"

df_leads_sorted["metric_4_score"] = df_leads_sorted["metric_4_score"].fillna(-1).apply(lambda x: f"{(x * 100):.2f}%")
df_leads_sorted.loc[df_leads_sorted["metric_4_score"] == "-100.00%", "metric_4_score"] = "N/A"

df_leads_sorted["metric_5_score"] = df_leads_sorted["metric_5_score"].fillna(-1).apply(lambda x: f"{(x * 100):.2f}%")
df_leads_sorted.loc[df_leads_sorted["metric_5_score"] == "-100.00%", "metric_5_score"] = "N/A"

df_leads_sorted["total_incentive_amt"] = df_leads_sorted["total_incentive_amt"].apply(lambda x: f"₱ {x:,.0f}")

# df_leads_sorted
df_leads_sorted.head()

<ipython-input-28-c9c54b94abe2>:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_leads_sorted["rank"] = df_leads_sorted.groupby(df_leads.columns[[3]].tolist()).cumcount() + 1


,report_date,employee_id,team_lead_name,account,metric_1_score,metric_2_score,metric_3_score,metric_4_score,metric_5_score,metric_1_incentive_amt,metric_2_incentive_amt,metric_3_incentive_amt,metric_4_incentive_amt,metric_5_incentive_amt,total_incentive_amt,rank
26,2024-08-31,TL27,TEAM LEAD NAME 27,Account 1,100.00%,0.00%,73.33%,73.33%,93.33%,1072,105,271,1905,1796,"₱ 5,149",1
25,2024-08-31,TL26,TEAM LEAD NAME 26,Account 1,100.00%,0.00%,66.67%,58.33%,75.00%,505,396,525,1298,1750,"₱ 4,474",2
24,2024-08-31,TL25,TEAM LEAD NAME 25,Account 1,100.00%,14.29%,69.23%,61.54%,53.85%,755,278,1508,420,233,"₱ 3,194",3
22,2024-08-31,TL23,TEAM LEAD NAME 23,Account 2,100.00%,16.67%,60.00%,20.00%,20.00%,1628,1909,1645,1538,1586,"₱ 8,306",1
23,2024-08-31,TL24,TEAM LEAD NAME 24,Account 2,92.96%,25.00%,50.00%,37.50%,0.00%,803,1609,644,580,1159,"₱ 4,795",2


In [29]:
# Double check the datatypes
df_leads_sorted.dtypes

,0
report_date,datetime64[ns]
employee_id,object
team_lead_name,object
account,category
metric_1_score,object
metric_2_score,object
metric_3_score,object
metric_4_score,object
metric_5_score,object
metric_1_incentive_amt,int64


# **Creating the Report Document in python-docx**

In [30]:
##### Creating the Document #####

from datetime import datetime
from docx import Document
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Pt
from docx.shared import Twips, Inches, Cm
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT



### Define functions ###

# Create a function to set table column widths
def set_table_column_widths(table, widths): #https://stackoverflow.com/questions/43051462/python-docx-how-to-set-cell-width-in-tables#:~:text=for%20cell%20in%20table.columns%5B0%5D.cells%3A%0A%20%20%20%20cell.width%20%3D%20Inches(0.5)
    for i, width in enumerate(widths):
        for cell in table.columns[i].cells:
            cell.width = width

# Create a function to set table borders
def set_table_borders(table):
    tbl = table._element
    tbl_pr = tbl.tblPr
    tbl_borders = OxmlElement("w:tblBorders")
    for border_name in ["top", "left", "bottom", "right", "insideH", "insideV"]:
        border = OxmlElement(f"w:{border_name}")
        border.set(qn("w:val"), "single")
        border.set(qn("w:sz"), "4")  # Border width
        border.set(qn("w:space"), "0")
        border.set(qn("w:color"), "000000")  # Border color
        tbl_borders.append(border)
    tbl_pr.append(tbl_borders)

# Function to make header rows bold
def set_header_cells_bold(hdr_cells):
    for cell in hdr_cells:
        for paragraph in cell.paragraphs:
            for run in paragraph.runs:
                run.font.bold = True



### Initialize creation of the document ###

doc = Document()

# Setting margins
sections = doc.sections
for section in sections:
    section.page_width = Cm(21)
    section.page_height = Cm(29.7)
    section.top_margin = Cm(2.54)
    section.bottom_margin = Cm(2.54)
    section.left_margin = Cm(2.54)
    section.right_margin = Cm(2.54)
    available_width = section.page_width - (section.left_margin + section.right_margin)

# Set default font style and size
style = doc.styles["Normal"]
font = style.font
font.name = "Arial"
font.size = Pt(8)

# Add document header
paragraph = doc.add_paragraph()
paragraph.add_run("REPORT TITLE LINE 1")
paragraph.add_run("\n")
paragraph.add_run("REPORT TITLE LINE 2")
paragraph.paragraph_format.space_before = Pt(0)
paragraph.paragraph_format.space_after = Pt(0)
paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
for run in paragraph.runs:
    run.font.name = "Lato"
    run.font.size = Pt(12)
    run.font.bold = True
    run.font.italic = False
    run.font.underline = False

# Add spacing after header
n = 3
while n > 0:
    paragraph = doc.add_paragraph("")
    paragraph.paragraph_format.space_before = Pt(0)
    paragraph.paragraph_format.space_after = Pt(0)
    n -= 1
    # for run in paragraph.runs:
    #     run.font.name = "Lato"
    #     run.font.size = Pt(12)
    #     run.font.bold = False
    #     run.font.italic = False
    #     run.font.underline = False

# Add body
report_month = input("Input report month (spell out the month completely; e.g., January): ").strip()
report_year = int(input("Input the report year: "))
paragraph = doc.add_paragraph()
paragraph.add_run("Lorem ipsum dolor sit amet, consectetur adipiscing elit. ")
paragraph.add_run(f"Etiam sapien mauris, sagittis non commodo sed, laoreet sed ligula {report_month} {report_year}.")
for run in paragraph.runs:
    run.font.name = "Lato"
    run.font.size = Pt(12)
    run.font.bold = False
    run.font.italic = False
    run.font.underline = False
paragraph = doc.add_paragraph("")  # Add a new paragraph for spacing
paragraph.paragraph_format.space_before = Pt(0)
paragraph.paragraph_format.space_after = Pt(0)

paragraph = doc.add_paragraph()
paragraph.add_run("Lorem ipsum dolor sit amet, consectetur adipiscing elit. Cras a aliquet magna. ")
paragraph.add_run("Morbi eget justo a lorem interdum consequat. Maecenas a dignissim tortor. Vivamus at lacus et felis iaculis lobortis.")
for run in paragraph.runs:
    run.font.name = "Lato"
    run.font.size = Pt(12)
    run.font.bold = False
    run.font.italic = False
    run.font.underline = False



'''
    Adding the tables to reflect the incentives and rankings for associates and team leads
'''

# Columns to include in the tables; maybe can implement this as a dictionary?
assocs_data_to_include = ["rank", "agent_name", "metric_1_score", "metric_2_score", "metric_3_score", "total_incentive_amt"]
assocs_table_headers = ["Rank", "Agent Name", "Metric 1", "Metric 2", "Metric 3", "Total Incentive Amount"]
leads_data_to_include = ["rank", "team_lead_name", "metric_1_score", "metric_2_score", "metric_3_score", "metric_4_score", "metric_5_score", "total_incentive_amt"]
leads_table_headers = ["Rank", "Team Lead Name", "Metric 1", "Metric 2", "Metric 3", "Metric 4", "Metric 5", "Total Incentive Amount"]

accounts = account_order # List of accounts to include

# Initialize variables to track changes
previous_account = ""
previous_team_lead = ""

for index, account in enumerate(accounts):
    current_account = accounts[index] if index < len(accounts) else None
    previous_account = accounts[index - 1] if index > 0 else None



    # Add new header when account changes
    if current_account != previous_account:
        if current_account != "Account 1":
            doc.add_page_break()
        else:
            # Add spacing
            n = 3
            while n > 0:
              paragraph = doc.add_paragraph("")  # Add a new paragraph for spacing
              paragraph.paragraph_format.space_before = Pt(0)
              paragraph.paragraph_format.space_after = Pt(0)
              for run in paragraph.runs:
                  run.font.name = "Lato"
                  run.font.size = Pt(12)
                  run.font.bold = False
                  run.font.italic = False
                  run.font.underline = False
              n -= 1
        paragraph = doc.add_paragraph(f"{current_account}")
        paragraph.paragraph_format.space_before = Pt(0)
        paragraph.paragraph_format.space_after = Pt(0)
        for run in paragraph.runs:
            run.font.name = "Lato"
            run.font.size = Pt(12)
            run.font.bold = True
            run.font.italic = False
            run.font.underline = True
        paragraph = doc.add_paragraph("")  # Add a new paragraph for spacing
        paragraph.paragraph_format.space_before = Pt(0)
        paragraph.paragraph_format.space_after = Pt(0)
        paragraph = doc.add_paragraph(f"{current_account} Associates")
        paragraph.paragraph_format.space_before = Pt(0)
        paragraph.paragraph_format.space_after = Pt(0)
        for run in paragraph.runs:
            run.font.name = "Lato"
            run.font.size = Pt(10)
            run.font.bold = True
            run.font.italic = True
            run.font.underline = False



    df_assocs_sorted_filtered = df_assocs_sorted[df_assocs_sorted["account"] == current_account]
    if not df_assocs_sorted_filtered.empty:
        for index, row in df_assocs_sorted_filtered.iterrows():
            current_team_lead = row["team_lead"]

            # Trigger when Team Lead changes
            if current_team_lead != previous_team_lead:
                n = 1
                while n > 0:
                  paragraph = doc.add_paragraph("")  # Add a new paragraph for spacing
                  paragraph.paragraph_format.space_before = Pt(0)
                  paragraph.paragraph_format.space_after = Pt(0)
                  n -= 1
                paragraph = doc.add_paragraph(f"Team Lead: {current_team_lead}")
                paragraph.paragraph_format.space_before = Pt(0)
                paragraph.paragraph_format.space_after = Pt(0)

                # Add a new table for incentive data for associates
                table = doc.add_table(rows=1, cols=len(assocs_data_to_include))
                set_table_borders(table)
                table.autofit = False
                table.allow_autofit = False
                table.alignment = WD_TABLE_ALIGNMENT.CENTER

                # Add the header row
                hdr_cells = table.rows[0].cells
                for i, column_name in enumerate(assocs_table_headers):
                    hdr_cells[i].text = column_name
                    hdr_cells[i].paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                    hdr_cells[i].paragraphs[0].paragraph_format.space_before = Pt(0)
                    hdr_cells[i].paragraphs[0].paragraph_format.space_after = Pt(0)
                set_header_cells_bold(hdr_cells) # Set header row to bold

            # Add employee data to the created table
            row_cells = table.add_row().cells
            for i, column_name in enumerate(assocs_data_to_include):
                row_cells[i].text = str(row[column_name])
                row_cells[i].paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                row_cells[i].paragraphs[0].paragraph_format.space_before = Pt(0)
                row_cells[i].paragraphs[0].paragraph_format.space_after = Pt(0)


            # # Set column widths
            # widths = [450000, 1440000, 957600, 957600, 957600, 957600] # measurement in EMU
            # set_table_column_widths(table, widths) # Not working, for some reason

            # Lazy implementation, convert to function with for loop:
            for cell in table.columns[0].cells:
                cell.width = 450000 # EMU equivalent of 1.25cm

            for cell in table.columns[1].cells:
                cell.width = 1440000 # EMU equivalent of 4cm

            for cell in table.columns[2].cells:
                cell.width = 957600 # EMU equivalent of 2.66cm

            for cell in table.columns[3].cells:
                cell.width = 957600

            for cell in table.columns[4].cells:
                cell.width = 957600

            for cell in table.columns[5].cells:
                cell.width = 957600


            previous_team_lead = current_team_lead



    df_leads_sorted_filtered = df_leads_sorted[df_leads_sorted["account"] == current_account]
    if not df_leads_sorted_filtered.empty:

        # Add spacing
        n = 2
        while n > 0:
          paragraph = doc.add_paragraph("")  # Add a new paragraph for spacing
          paragraph.paragraph_format.space_before = Pt(0)
          paragraph.paragraph_format.space_after = Pt(0)
          n -= 1
        paragraph = doc.add_paragraph(f"{current_account} Team Leads")
        paragraph.paragraph_format.space_before = Pt(0)
        paragraph.paragraph_format.space_after = Pt(0)
        for run in paragraph.runs:
            run.font.name = "Lato"
            run.font.size = Pt(10)
            run.font.bold = True
            run.font.italic = True
            run.font.underline = False
        paragraph = doc.add_paragraph("")  # Add a new paragraph for spacing
        paragraph.paragraph_format.space_before = Pt(0)
        paragraph.paragraph_format.space_after = Pt(0)

        counter = 0
        for index, row in df_leads_sorted_filtered.iterrows():
            if counter == 0:
                # Add a new table for incentive data for leads
                table = doc.add_table(rows=1, cols=len(leads_data_to_include))
                set_table_borders(table)
                table.autofit = False
                table.allow_autofit = False
                table.alignment = WD_TABLE_ALIGNMENT.CENTER

                # Add the header row
                hdr_cells = table.rows[0].cells
                for i, column_name in enumerate(leads_table_headers):
                    hdr_cells[i].text = column_name
                    hdr_cells[i].paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                    hdr_cells[i].paragraphs[0].paragraph_format.space_before = Pt(0)
                    hdr_cells[i].paragraphs[0].paragraph_format.space_after = Pt(0)
                set_header_cells_bold(hdr_cells) # Set header row to bold

            # Add employee data to the created table
            row_cells = table.add_row().cells
            for i, column_name in enumerate(leads_data_to_include):
                row_cells[i].text = str(row[column_name])
                row_cells[i].paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                row_cells[i].paragraphs[0].paragraph_format.space_before = Pt(0)
                row_cells[i].paragraphs[0].paragraph_format.space_after = Pt(0)

            # # Set column widths
            # widths = [450000, 748800, 748800, 748800, 748800, 748800, 748800, 748800]
            # set_table_column_widths(table, widths) # Not working, for some reason

            # Lazy implementation, convert to function with for loop:
            for cell in table.columns[0].cells:
                cell.width = 450000

            for cell in table.columns[1].cells:
                cell.width = 748800

            for cell in table.columns[2].cells:
                cell.width = 748800

            for cell in table.columns[3].cells:
                cell.width = 748800

            for cell in table.columns[4].cells:
                cell.width = 748800

            for cell in table.columns[5].cells:
                cell.width = 748800

            for cell in table.columns[6].cells:
                cell.width = 748800

            for cell in table.columns[7].cells:
                cell.width = 748800


            counter += 1





# report_month_num = datetime.strptime(report_month, "%B").month
# if len(str(report_month_num)) == 1:
#     report_month_num = "0" + str(report_month_num)
#     doc.save(f"{report_year}-{report_month_num} - <Report Title Here>.docx")
# else:
#     report_month_num = str(report_month_num)
#     doc.save(f"{report_year}-{report_month_num} - <Report Title Here>.docx")

# print(f"Report for {report_month} {report_year} successfully created!")
# print(f"Document saved as '{report_year}-{report_month_num} - <Report Title Here>.docx'")

doc.save("report.docx")
print("Report successfully created!")
print("Document saved as report.docx")

Input report month (spell out the month completely; e.g., January): October
Input the report year: 2024
Report successfully created!
Document saved as report.docx
